In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm
from lightgbm import LGBMRegressor
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
data=pd.read_csv('./data/train.csv', encoding='UTF-8')

In [3]:
data

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,0,5011000595017300,2871000192069300,음반,3
1,1,4148000690043300,5011000264024400,문화컨텐츠,3
2,2,5011000078068400,1120000007005400,농산물,3
3,3,4127100048006400,5011000587019400,기타식품,7
4,4,5011000078068400,2823700010076300,농산물,3
...,...,...,...,...,...
31679,31679,4471000290087200,5011000213073200,스포츠잡화,3
31680,31680,1129000014045300,5011000319087100,스마트디바이스,4
31681,31681,1129000014045300,5011000263065200,스마트디바이스,6
31682,31682,4127300065073100,5011000264061200,지갑,7


In [4]:
data['round_send']=data['송하인_격자공간고유번호'].round(-10)
data['round_rec']=data['수하인_격자공간고유번호'].round(-10)

In [5]:
data

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,round_send,round_rec
0,0,5011000595017300,2871000192069300,음반,3,5011000000000000,2871000000000000
1,1,4148000690043300,5011000264024400,문화컨텐츠,3,4148000000000000,5011000000000000
2,2,5011000078068400,1120000007005400,농산물,3,5011000000000000,1120000000000000
3,3,4127100048006400,5011000587019400,기타식품,7,4127100000000000,5011000000000000
4,4,5011000078068400,2823700010076300,농산물,3,5011000000000000,2823700000000000
...,...,...,...,...,...,...,...
31679,31679,4471000290087200,5011000213073200,스포츠잡화,3,4471000000000000,5011000000000000
31680,31680,1129000014045300,5011000319087100,스마트디바이스,4,1129000000000000,5011000000000000
31681,31681,1129000014045300,5011000263065200,스마트디바이스,6,1129000000000000,5011000000000000
31682,31682,4127300065073100,5011000264061200,지갑,7,4127300000000000,5011000000000000


In [6]:
a=list(set(data.round_send.to_list()))

In [7]:
b=list(set(data.round_rec.to_list()))

In [8]:
sums=list(set(a+b))

In [9]:
matrix = []
for i,j in enumerate(sums):
    vector = [j,i]
    matrix.append(vector)

In [10]:
matrix = dict(matrix)

In [14]:
matrix[4148000000000000]


1

In [63]:
for i in range(len(data)):
    data.loc[i,'send_dict']=str(matrix[data['round_send'][i]])
    data.loc[i,'rec_dict']=str(matrix[data['round_rec'][i]])

In [64]:
dummies_data=pd.get_dummies(data)

In [65]:
data

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,round_send,round_rec,send_dict,rec_dict
0,0,5011000595017300,2871000192069300,음반,3,5011000000000000,2871000000000000,0,136
1,1,4148000690043300,5011000264024400,문화컨텐츠,3,4148000000000000,5011000000000000,1,0
2,2,5011000078068400,1120000007005400,농산물,3,5011000000000000,1120000000000000,0,10
3,3,4127100048006400,5011000587019400,기타식품,7,4127100000000000,5011000000000000,3,0
4,4,5011000078068400,2823700010076300,농산물,3,5011000000000000,2823700000000000,0,32
...,...,...,...,...,...,...,...,...,...
31679,31679,4471000290087200,5011000213073200,스포츠잡화,3,4471000000000000,5011000000000000,15,0
31680,31680,1129000014045300,5011000319087100,스마트디바이스,4,1129000000000000,5011000000000000,174,0
31681,31681,1129000014045300,5011000263065200,스마트디바이스,6,1129000000000000,5011000000000000,174,0
31682,31682,4127300065073100,5011000264061200,지갑,7,4127300000000000,5011000000000000,155,0


In [66]:
dummies_data

,index,송하인_격자공간고유번호,수하인_격자공간고유번호,운송장_건수,round_send,round_rec,물품_카테고리_DIY자재/용품,물품_카테고리_PC,물품_카테고리_PC주변기기,물품_카테고리_가공식품,...,rec_dict_90,rec_dict_91,rec_dict_92,rec_dict_93,rec_dict_94,rec_dict_95,rec_dict_96,rec_dict_97,rec_dict_98,rec_dict_99
0,0,5011000595017300,2871000192069300,3,5011000000000000,2871000000000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,4148000690043300,5011000264024400,3,4148000000000000,5011000000000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,5011000078068400,1120000007005400,3,5011000000000000,1120000000000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,4127100048006400,5011000587019400,7,4127100000000000,5011000000000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,5011000078068400,2823700010076300,3,5011000000000000,2823700000000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31679,31679,4471000290087200,5011000213073200,3,4471000000000000,5011000000000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31680,31680,1129000014045300,5011000319087100,4,1129000000000000,5011000000000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31681,31681,1129000014045300,5011000263065200,6,1129000000000000,5011000000000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31682,31682,4127300065073100,5011000264061200,7,4127300000000000,5011000000000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
feature=list(dummies_data.columns)[6:]

In [72]:
len(feature)

588

In [74]:
dummies_data[feature].head()

,물품_카테고리_DIY자재/용품,물품_카테고리_PC,물품_카테고리_PC주변기기,물품_카테고리_가공식품,물품_카테고리_가방,물품_카테고리_건강식품,물품_카테고리_건강용품,물품_카테고리_게임기/타이틀,물품_카테고리_계절가전,물품_카테고리_골프,...,rec_dict_90,rec_dict_91,rec_dict_92,rec_dict_93,rec_dict_94,rec_dict_95,rec_dict_96,rec_dict_97,rec_dict_98,rec_dict_99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
raw_x=dummies_data[feature]
raw_y=dummies_data[['운송장_건수']]

train_x, test_x, train_y, test_y = train_test_split(raw_x, raw_y, test_size=0.2, random_state=42)

train_ds = lightgbm.Dataset(train_x, label = train_y) 
test_ds = lightgbm.Dataset(test_x, label = test_y) 

print(train_y.max())
print(test_y.max())

운송장_건수    413
dtype: int64
운송장_건수    179
dtype: int64


In [90]:
params = { 
          'learning_rate': 0.082, 
          'max_depth': 5, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 50, 
          'feature_fraction': 0.8, 
          'bagging_fraction': 0.6, 
          'bagging_freq': 2,
          'min_data_in_leaf':2,
         }

model = lightgbm.train(params, train_ds, 1500, test_ds, verbose_eval=100, early_stopping_rounds=200)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000394 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1128
[LightGBM] [Info] Number of data points in the train set: 25347, number of used features: 564
[LightGBM] [Info] Start training from score 4.880972
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

In [92]:
mean_squared_error(test_y,model.predict(test_x))

34.65843394949962

In [96]:
# train_x, test_x, train_y, test_y 

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import mean_absolute_error

  # MAPE Metric
def mean_absolute_percentage_error(test_y, y_pred):
  test_y, y_pred = np.array(test_y), np.array(y_pred)
  return np.mean(np.abs((test_y - y_pred) / test_y)) * 100

  # 탐색 대상 함수 (XGBRegressor)
def lgbm_cv(max_depth,learning_rate, n_estimators, 
            min_child_weight, subsample
            ,colsample_bytree, silent=True, nthread=-1):
  

      # 모델 정의
  model = LGBMRegressor(max_depth=int(max_depth),
                        learning_rate=learning_rate,
                        n_estimators=int(n_estimators),
                        # gamma=gamma,
                        min_child_weight=min_child_weight,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree, 
                        nthread=nthread)
      # 모델 훈련
  model.fit(train_x, train_y)

      # 예측값 출력
  y_pred = model.predict(test_x)

  rmse = np.sqrt(mean_squared_error(test_y, y_pred))
  r2 = r2_score(test_y, y_pred)
  mape = mean_absolute_percentage_error(test_y, y_pred)
  mean_absolute_error(test_y, y_pred)

      # 오차 최적화로 사용할 metric 반환
  return r2

In [98]:
# 실험해보고자하는 hyperparameter 집합
pbounds = {'max_depth': (1, 15),
           'learning_rate': (0.01, 0.5),
           'n_estimators': (50, 1000),
          #  'gamma': (0, 100),/
           'min_child_weight': (0, 3),
           'subsample': (0.1, 0.99),
           'colsample_bytree' :(0.1, 0.99)
           }

  # verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
  # random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제 
bo=BayesianOptimization(f=lgbm_cv, pbounds=pbounds, verbose=2, random_state=42 )

  # init_points :  초기 Random Search 갯수
  # n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
  # acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
  # xi : exploration 강도 (기본값은 0.0)
bo.maximize(init_points=2, n_iter=10, acq='ei', xi=0.01)

  # 찾은 파라미터 값 확인
print(bo.max)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
| 1         | 0.1045    | 0.4333    | 0.4759    | 11.25     | 1.796     | 198.2     | 0.2388    |
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
| 2         | 0.06992   | 0.1517    | 0.4344    | 9.416     | 2.124     | 69.56     | 0.9632    |
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
| 3         | 0.09824   | 0.3073    | 0.4242    | 11.18     | 2.942     | 197.9     | 0.3008    |
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
| 4         | 0.1073    | 0.7692    | 0.3094    | 10.8

In [99]:
lgbm_ff = LGBMRegressor(random_state=42, learning_rate=0.304, metric = 'mae',
                      n_estimators=202,max_depth=8,min_child_weight=0.0672)

evals = [(test_x, test_y)]

lgbm_ff.fit(train_x, train_y).score(test_x, test_y)

0.10891221411752128

In [100]:
mean_squared_error(test_y,lgbm_ff.predict(test_x))

34.10126033004438